In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile

import pandas as pd
import numpy as np

#jobName = '201216_Train_pilot'
jobName = '201216_Train_split1'
x, y = LibData.GetXY_s_o(jobName)

#l_x, l_y = LibData.GetSplited5(x,y)


     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 1.3MB 5.9MB/s 


In [ ]:
from sklearn.model_selection import train_test_split

xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import datetime
import LibData

# 가장 좋은 결과
트리수 = 700
최소갯수inLeaf = 2
최대피쳐수=26

def Fit(xTrain, yTrain, xVal):
    model = RandomForestRegressor(n_estimators =트리수, max_features=최대피쳐수, min_samples_leaf=최소갯수inLeaf)
    model.fit(xTrain, yTrain)
    y_pred = pd.Series(model.predict(xVal), index=xVal.index)
    return model, y_pred

start = datetime.datetime.now()
print("start time : " + start.strftime('%Y-%m-%d %H:%M:%S'))

#model_f, y_pred_f = Fit(xTrain, yTrain['pv_fee'], xVal)
middle = datetime.datetime.now()
print("dur1 : " + str(middle - start))

model_c, y_pred_c = Fit(xTrain, yTrain['pv_claim'], xVal)
end = datetime.datetime.now()
print("dur2 : " + str(end-middle))
print("end time : " + end.strftime('%Y-%m-%d %H:%M:%S'))

LibUtils.ELoop()

start time : 2020-12-23 23:39:56
dur1 : 0:00:00.000964
dur2 : 0:09:19.834364
end time : 2020-12-23 23:49:16
start empty loop.

KeyboardInterrupt: 

In [ ]:

# 0 이하로 측정한 것은 0으로
#y_pred_f_modi = y_pred_f.apply(lambda x: 0.0 if x<0.0 else x)
y_pred_c_modi = y_pred_c.apply(lambda x: 0.0 if x<0.0 else x)
#print(LibUtils.GetScores(yVal['pv_fee'], y_pred_1))
#print(LibUtils.GetScores(yVal['pv_fee'], y_pred_1_modi))
print(LibUtils.GetScores(yVal['pv_claim'], y_pred_c))
print(LibUtils.GetScores(yVal['pv_claim'], y_pred_c_modi))
# --> 0이하로 예측한 것이 없음



[159.74914488149375, 0.1925847484080104, 0.8763417907379579]
[159.74914488149375, 0.1925847484080104, 0.8763417907379579]


In [ ]:
importance = pd.concat([pd.Series(xVal.columns), pd.Series(model_c.feature_importances_)], axis=1)
importance.columns=['name','rate']
importance = importance.sort_values('rate', ascending=False)

# 중요도가 1% 이상인 것만 고름, 20개 선정
picked = np.array(importance.loc[importance['rate'] > 0.01]['name'])
picked.shape

In [ ]:
xTrain_picked = xTrain[picked]
xVal_picked = xVal[picked]

최대피쳐수 = len(picked)

# model_f_picked = RandomForestRegressor(n_estimators=트리수, max_features=최대피쳐수, min_samples_leaf=최소갯수inLeaf)
# model_f_picked.fit(xTrain_picked, yTrain['pv_cla'])
# y_pred_fee = pd.Series(model_fee_picked.predict(xVal_picked), index=xVal.index)

model_c_picked = RandomForestRegressor(n_estimators=트리수, max_features=최대피쳐수, min_samples_leaf=최소갯수inLeaf)
model_c_picked.fit(xTrain_picked, yTrain['pv_claim'])

In [ ]:
y_pred_c = pd.Series(model_c_picked.predict(xVal_picked), index=xVal_picked.index)

#print(LibUtils.GetScores(yVal['pv_fee'], y_pred_f))
print(LibUtils.GetScores(yVal['pv_claim'], y_pred_c))

# 결과 좋아짐
# 처음 : [159.74914488149375, 0.1925847484080104, 0.8763417907379579]
# 이번 : [151.7813710244733, 0.18297924019221043, 0.8762580680631261]

[151.7813710244733, 0.18297924019221043, 0.8762580680631261]


In [ ]:
# 주요변수
import matplotlib.pyplot as plt

n_feature = xTrain.shape[1]
index = np.arange(n_feature)

plt.barh(index, model_1.feature_importances_, align='center')
plt.yticks(index, xTrain.columns)
plt.ylim(-1,20, n_feature)
plt.xlabel('feature importance', size=15)
plt.ylabel('feature', size=15)
plt.show()

In [3]:
########## 스케일러 적용 테스트

jobName = '201216_Train_split1'

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=700, max_features=26, min_samples_leaf=2)

# 0. base
x, y = LibData.GetXYBase(jobName)
l_x, l_y = LibData.GetSplited(x,y,5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('0_base', [str(sum(maes)/5), maes])

# 1. scaler
x, y = LibData.GetXY_s(jobName)
l_x, l_y = LibData.GetSplited(x,y,5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('1_scaler', [str(sum(maes)/5), maes])

# 2. oneoht
x, y = LibData.GetXY_o(jobName)
l_x, l_y = LibData.GetSplited(x,y,5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('2_onehot', [str(sum(maes)/5), maes])

# 3. scaler + onehot
x, y = LibData.GetXY_s_o(jobName)
l_x, l_y = LibData.GetSplited(x,y,5)
preds = LibML.GetPredCV(l_x, l_y, model, 'pv_claim')
maes = LibML.GetMaesCV(l_y, preds, 'pv_claim')
print('3_scaler+onehot', [str(sum(maes)/5), maes])


start learning 1 2020-12-27 14:33... end learning 2020-12-27 14:42
start learning 2 2020-12-27 14:42... end learning 2020-12-27 14:52
start learning 3 2020-12-27 14:52... end learning 2020-12-27 15:02
start learning 4 2020-12-27 15:02... end learning 2020-12-27 15:12
start learning 5 2020-12-27 15:12... end learning 2020-12-27 15:22
0_base ['124.76428102195499', [126.13084643204664, 122.18306132710919, 126.54096945159336, 123.95534090384822, 125.01118699517751]]
start learning 1 2020-12-27 15:22... end learning 2020-12-27 15:33
start learning 2 2020-12-27 15:33... end learning 2020-12-27 15:43
start learning 3 2020-12-27 15:43... end learning 2020-12-27 15:53
start learning 4 2020-12-27 15:53... end learning 2020-12-27 16:04
start learning 5 2020-12-27 16:04... end learning 2020-12-27 16:14
1_scaler ['124.46363751119989', [125.321443366187, 126.27625925354164, 125.72619981247372, 120.91348824043314, 124.08079688336393]]
start learning 1 2020-12-27 16:14... end learning 2020-12-27 16:22